<a href="https://colab.research.google.com/github/preetamjumech/AI-Agent-RouteLLM-llama-index/blob/main/RouteLLM_Summarization_and_RAG_16_01_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install llama-index==0.12.9 llama-index-llms-ollama==0.5.0 llama-index-embeddings-huggingface==0.4.0 llama-index-llms-groq==0.3.1 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [3]:
import os

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
from llama_index.core import SimpleDirectoryReader

In [15]:
documents = SimpleDirectoryReader(input_files=["/content/attention-is-all-you-need.pdf"]).load_data()
print(type(documents))

<class 'list'>


In [16]:
documents[0]

Document(id_='523cc231-1ad4-4e60-baa6-1438c0fd3cac', embedding=None, metadata={'page_label': '1', 'file_name': 'attention-is-all-you-need.pdf', 'file_path': '/content/attention-is-all-you-need.pdf', 'file_type': 'application/pdf', 'file_size': 2201700, 'creation_date': '2025-01-16', 'last_modified_date': '2025-01-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAida

In [17]:
len(documents)

15

In [19]:
from llama_index.core.node_parser import SentenceSplitter

In [20]:
splitter = SentenceSplitter(
    chunk_size= 2000,
    chunk_overlap=20
)

In [21]:
nodes = splitter.get_nodes_from_documents(documents)
print(type(nodes))

<class 'list'>


In [22]:
len(nodes)

15

In [24]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [25]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

In [44]:
# %%
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

# Check if api_key is set and valid
if not api_key:
  raise ValueError("GROQ_API_KEY is not set in user data. Please set it using `userdata.set('GROQ_API_KEY', 'YOUR_API_KEY')`.")
# %%
Settings.llm = Groq(model = "llama-3.1-8b-instant", api_key = api_key)
# %%

In [46]:
groq_llm = Groq(model = "llama-3.1-8b-instant", api_key = api_key)

In [47]:
Settings.llm = groq_llm

In [48]:
Settings.embed_model = HuggingFaceEmbedding()

In [28]:
from llama_index.core import SummaryIndex, VectorStoreIndex

In [29]:
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [49]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True
)

vector_query_engine = vector_index.as_query_engine()

In [50]:
from llama_index.core.tools import QueryEngineTool

In [51]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine = summary_query_engine,
    description = "Useful for summarization related to given context"
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine = vector_query_engine,
    description = "Useful for retrieving specific context from the given context based on given question"
)

In [52]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

In [53]:
query_engine = RouterQueryEngine(
    selector = LLMSingleSelector.from_defaults(),
    query_engine_tools = [summary_tool, vector_tool],
    verbose = True
)

In [54]:
response = query_engine.query("What is the summary of the document?")
print(response)

Selecting query engine 0: The question 'What is the summary of the document?' implies a need to condense the information into a brief overview, which aligns with the description of choice 1 as 'Useful for summarization related to given context'..
The document presents a new approach to sequence transduction models, which are used in tasks such as machine translation and language modeling. It introduces a novel model called the Transformer, which relies on self-attention mechanisms to draw global dependencies between input and output sequences. The Transformer model is shown to be effective in various tasks, achieving state-of-the-art results in some cases, and generalizes well to other tasks such as English constituency parsing. The document also explores variations of the Transformer model and its applications in natural language processing.


In [55]:
response = query_engine.query("What is the model architechture mentioned in the document?")
print(response)

Selecting query engine 1: The question is asking for a specific piece of information (the model architecture) from the given context (the document), which matches option 2..
The Transformer architecture is a model that uses self-attention to process input sequences. It consists of an encoder and a decoder, with multiple layers of self-attention and feed-forward neural networks. The Transformer architecture is designed to handle long-range dependencies in input sequences and has been shown to be effective in various natural language processing tasks, including machine translation.
